### 배치 정규화

- 입력 데이터의 분포를 정규화(Normalization)하여 학습을 더 안정적이고 빠르게 진행하도록 하는 기법

#### 하는 이유
- 딥러닝에서 각 층의 출력이 다음 층의 입력이 되는데, 이 입력의 분포가 학습 중에 계속 바뀌면 모델이 학습을 제대로 하지 못함
- 입력 데이터의 분포가 너무 크거나 작으면, 역전파 과정에서 기울기(gradient)가 0에 수렴하거나 너무 커지는 문제가 발생
- 가중치가 적절한 범위에서 학습되므로, 학습률을 더 크게 설정

### 가중치 초기화

#### 제이비어 초기화
- 시그모이드나 Tanh 같은 비선형 함수는 입력 값이 크면 기울기 소실 문제가 발생하기 쉬움.
- 가중치 초기화를 잘못하면 뉴런들이 비활성화(출력이 0 또는 1로 수렴)되거나, 반대로 너무 커서 학습이 어려워짐.
- 초기화를 사용하면 입력과 출력의 분산이 비슷하게 유지되어 적절한 범위에서 학습 가능.

#### 허 초기화
- 제이비어 초기화는 대칭적인 함수에 적합하지만, Relu는 대칭 함수가 아님.
- 출력보다는 입력 노드만 고려하여 ReLU가 절반의 뉴런을 0으로 만드는 특성을 반영해서, 활성화된 뉴런의 분산을 적절하게 유지.

#### 직교 초기화
- RNN의 기울기 소실 문제 해결용.
- SVD를 사용해 모든 행,열 벡터와 직교인 단위벡터 행렬을 만드는 방법으로, 이 단위벡터의 절댓값은 1이기에 아무리 곱해도 기울기 폭주/소실 일어나지 않음

In [4]:
from torch import nn


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(1, 2),
            nn.Sigmoid()
        )
        self.fc = nn.Linear(2, 1)
        self._init_weights()

    def _init_weights(self):
        nn.init.xavier_uniform_(self.layer[0].weight)
        self.layer[0].bias.data.fill_(0.01)

        nn.init.xavier_uniform_(self.fc.weight)
        self.fc.bias.data.fill_(0.01)


model = Net()

In [5]:
from torch import nn


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(1, 2),
            nn.Sigmoid()
        )
        self.fc = nn.Linear(2, 1)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            nn.init.constant_(module.bias, 0.01)
        print(f"Apply : {module}")


model = Net()

Apply : Linear(in_features=1, out_features=2, bias=True)
Apply : Sigmoid()
Apply : Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): Sigmoid()
)
Apply : Linear(in_features=2, out_features=1, bias=True)
Apply : Net(
  (layer): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): Sigmoid()
  )
  (fc): Linear(in_features=2, out_features=1, bias=True)
)


### 정칙화
머신러닝 모델이 과적합 되는 것을 방지하기 위해 사용하는 기법
모델이 암기가 아니라 일반화하도록 함.


#### L1 정칙화
L1은 가중치의 절댓값 합(|W|)을 패널티로 추가하는 방식
- 일부 가중치를 정확히 0으로 만듦
- 모델이 중요하지 않은 특성을 자동으로 제거


In [ ]:

for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)

    _lambda = 0.5
    l1_loss = sum(p.abs().sum() for p in model.parameters())

    loss = criterion(output, y) + _lambda * l1_loss

사용법이라 데이터같은거 없음

#### L2 정칙화
L2 정칙화는 가중치의 제곱 합(W^2)을 패널티로 추가하는 방식

- 모든 가중치를 작게 만들지만 0으로는 만들지 않음
- 가중치가 작아져서 과적합 방지에 효과적

#### 가중치 감쇠
L2와 같은 용어로 사용은 하지만, 실상은 손실 함수 자체에 규제 항을 추가하는 방식을 뜻한다.
파이토치에서는 L2와 같은 용어로서 사용을 하고, 같은 방식으로 수행한다.

In [ ]:
for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)

    _lambda = 0.5
    l2_loss = sum(p.pow(2.0).sum() for p in model.parameters())

    loss = criterion(output, y) + _lambda * l2_loss

#### 모멘텀
그래디언트 디센트의 변형 중 하나.

- 최적화 함수의 momentum 하이퍼 파라미터를 사용해서 적용

#### 엘라스틱 넷
Elastic Net은 L1과 L2 정칙화를 결합한 기법으로, 두 방식의 장점을 모두 활용

- L1 정칙화의 희소성 + L2 정칙화의 안정성 결합

#### 드롭다운
학습 중 특정 뉴런을 확률적으로 제거하여 과적합을 방지

- 학습 과정에서 일부 뉴런을 랜덤하게 0으로 설정하여 특정 뉴런에 의존하는 것을 방지
- 다양한 뉴런 조합을 학습함으로써 더 일반화된 모델을 생성

In [7]:
from torch import nn


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10, 10)
        self.dropout = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(10, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        return x

#### 그래디언트 클리핑
그래디언트 클리핑은 기울기의 크기를 일정 값 이하로 제한하여 폭발을 방지
- 임계값(threshold, T)을 설정하고, 기울기가 그 이상 커지면 스케일을 조정하는 방식으로 작동
- 각 그래디언트 값이 임계값 T을 초과하면, 최대값을 T로 제한하는 방법도 있다.

In [ ]:
for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)
    
    optimizer.zero_grad()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
 
    optimizer.step()